In [ ]:
import torch
from torch import nn
from typing import Optional, Literal

import torch.nn.functional as F

class ComplexFIR(nn.Module):
    def __init__(
        self,
        m: int,
        coeff: Optional[torch.Tensor] = None,
        init: Literal["zeros", "delta", "central_delta"] = "delta",
        trainable: bool = True,
        dtype: torch.dtype = torch.complex64,
    ):
        super().__init__()

        if not isinstance(m, int):
            raise TypeError("m must be int")
        if m <= 0:
            raise ValueError("m must be >= 1")

        if coeff is not None:
            if not isinstance(coeff, torch.Tensor):
                raise TypeError("coeff must be a torch.Tensor")
            if coeff.ndim != 1:
                raise ValueError("coeff must be 1D tensor")
            if coeff.numel() != m:
                raise ValueError("coeff length must be m")
            if not torch.is_complex(coeff):
                raise TypeError("coeff must be complex")
            h = coeff.to(dtype=dtype)
        else:
            h = torch.zeros(m, dtype=dtype)
            if init == "zeros":
                pass
            elif init == "delta":
                h[0] = 1
            elif init == "central_delta":
                h[m // 2] = 1
            else:
                raise ValueError("unknown init")

        if trainable:
            self.h = nn.Parameter(h)
        else:
            self.register_buffer("h", h)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.ndim != 2:
            raise ValueError("x must have shape (B, T)")
        if not torch.is_complex(x):
            raise TypeError("x must be a complex tensor")

        _, T = x.shape
        h = self.h
        m = h.numel()

        if T == 0:
            return x
        if m == 1:
            return x * h[0]
        pad = m - 1
        x1 = x.unsqueeze(1)
        x_pad = F.pad(x1, (pad, 0))
        y = F.conv1d(x_pad, h.flip(0).view(1, 1, m), bias=None).squeeze(1)
        return y

In [61]:
torch.eye(5)[1, :].flip(0)

tensor([0., 0., 0., 1., 0.])

In [64]:
x = torch.randn(2, 16, dtype=torch.complex64, requires_grad=False)
fir = ComplexFIR(m=5, trainable=True)
loss = fir(x).abs().mean()
loss.backward()
assert fir.h.grad is not None


In [53]:
b = a.reshape(20, 2, -1)
b.shape, b.permute(1, 0, 2).contiguous().shape

(torch.Size([20, 2, 250]), torch.Size([2, 20, 250]))

In [65]:
torch.zeros(4)

tensor([0., 0., 0., 0.])

In [70]:
import os
os.chdir(r"D:\github_projects\dpd-baselines\src")

In [71]:
import torch
import pytest

from dpd_baselines.blocks.polinomials import ChebPoly 

def test_identity_init_returns_ones():
    x = torch.linspace(-1.0, 1.0, 11).repeat(2, 1)  # (B,T), real
    m = ChebPoly(order=5, coeff=None, trainable=False)
    g = m(x)
    expected = torch.ones_like(x, dtype=torch.complex64)
    torch.testing.assert_close(g, expected)

def test_order1_is_constant():
    x = torch.randn(3, 7)
    c0 = torch.tensor([2.0 + 1.0j], dtype=torch.complex64)
    m = ChebPoly(order=1, coeff=c0, trainable=False)
    g = m(x)
    expected = (2.0 + 1.0j) * torch.ones_like(x, dtype=torch.complex64)
    torch.testing.assert_close(g, expected)

def test_order2_matches_linear_formula():
    # g(x) = c0*T0(x) + c1*T1(x) = c0 + c1*x
    x = torch.linspace(-1.0, 1.0, 9).repeat(2, 1)
    coeff = torch.tensor([1.0 + 2.0j, 3.0 - 1.0j], dtype=torch.complex64)
    m = ChebPoly(order=2, coeff=coeff, trainable=False)
    g = m(x)
    expected = coeff[0] + coeff[1] * x
    torch.testing.assert_close(g, expected)

def test_order3_matches_T2():
    # T0=1, T1=x, T2=2x^2-1
    x = torch.linspace(-1.0, 1.0, 13).repeat(1, 1)
    coeff = torch.tensor([1.0 + 0.0j, 0.5 + 0.2j, -0.25 + 0.1j], dtype=torch.complex64)
    m = ChebPoly(order=3, coeff=coeff, trainable=False)
    g = m(x)
    T2 = 2.0 * x**2 - 1.0
    expected = coeff[0] + coeff[1] * x + coeff[2] * T2
    torch.testing.assert_close(g, expected)

def test_gradient_flows_to_coeff():
    x = torch.linspace(-1.0, 1.0, 31).repeat(4, 1)
    m = ChebPoly(order=4, coeff=None, trainable=True)
    g = m(x)
    loss = g.abs().mean()  # real scalar
    loss.backward()
    assert m.coeff.grad is not None
    assert m.coeff.grad.shape == m.coeff.shape

def test_rejects_complex_input():
    x = torch.randn(2, 5, dtype=torch.complex64)
    m = ChebPoly(order=3, coeff=None, trainable=False)
    with pytest.raises(TypeError):
        _ = m(x)
